In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
from splinter.exceptions import ElementDoesNotExist
from splinter import Browser
from selenium import webdriver
import pandas as pd
import os
import time

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver/

# NASA Mars News

In [3]:
mars_dict = {}
# visit the NASA Mars News site and scrape headlines
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)


url = "https://mars.nasa.gov/news/"
browser.visit(url)
# import time
time.sleep(5)

#using bs to write it into html
html = browser.html
soup = bs(html,"html.parser")

In [4]:
# save the most recent article, title
news_title  = soup.find("div",class_="content_title").text
news_paragraph = soup.find("div", class_="article_teaser_body").text
print(f"Title: {news_title}")
print(f"Para: {news_paragraph}")

Title: Six Things About Opportunity's Recovery Efforts
Para: The global dust storm on Mars could soon let in enough sunlight for the Opportunity rover to recharge.


# JPL Mars Space Images - Featured Image

In [5]:
# Visit the JPL Mars URL
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

# Setting up splinter
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

#visiting the page
browser.visit(url)

# Moving through the pages
time.sleep(1)
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(1)
browser.click_link_by_partial_text('more info')
time.sleep(1)


In [6]:
#using bs to write it into html
html = browser.html
soup = bs(html, "html.parser")

# Get featured image
results = soup.find("article")
extension = results.find("figure", class_="lede").a["href"]
link = "https://www.jpl.nasa.gov"
featured_image_url = link + extension
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA18886_hires.jpg


# Mars Weather

In [7]:
#Get mars weather, THE INSTRUCTIONS SAY SPECIFICALLY TO SCRAPE THE DATA

url= "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
html = browser.html
soup = bs(html, "html.parser")
# twitter_soup = bs(twitter_response, 'html.parser')

In [8]:

# #temp = soup.find('div', attrs={"class": "tweet", "data-name": "Mars Weather"})
mars_weather = soup.find_all("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text")
print(mars_weather)

# mars_weather_url = 'https://twitter.com/marswxreport?lang=en'
# browser.visit(mars_weather_url)
# time.sleep(1)
# mars_weather_html = browser.html
# mars_weather_soup = bs(mars_weather_html, 'html.parser')

# tweets = mars_weather_soup.find('ol', class_='stream-items')
# mars_weather = tweets.find('p', class_="tweet-text").text
# print(mars_weather)
# #temp


[<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">Sol 2142 (2018-08-15), high -10C/14F, low -71C/-95F, pressure at 8.65 hPa, daylight 05:28-17:41</p>, <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">Sol 2141 (2018-08-14), high -17C/1F, low -66C/-86F, pressure at 8.63 hPa, daylight 05:27-17:41</p>, <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">Clearing skies following the Mars dust storm are evident in these images by <a class="twitter-atreply pretty-link js-nav" data-mentioned-user-id="2711847374" dir="ltr" href="/TerryLovejoy66"><s>@</s><b>TerryLovejoy66</b></a><a class="twitter-timeline-link u-hidden" data-pre-embedded="true" dir="ltr" href="https://t.co/tYjUY1UjFn">pic.twitter.com/tYjUY1UjFn</a></p>, <p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="en">So

In [9]:
for i in range(30):
    tweets = mars_weather[i].text
#         global weather
    if "Sol " in tweets:
        weather =tweets
#         print(tweets)
        break
mar_weather=weather
mars_dict["mar_weather"] = mar_weather
print (mar_weather)

Sol 2142 (2018-08-15), high -10C/14F, low -71C/-95F, pressure at 8.65 hPa, daylight 05:28-17:41


# Mars Facts

In [10]:
#Mars Facts....visit webpage, use pandas to scrape the page for facts, 
#convert pandas table to html table string. 
mars_facts_url = "https://space-facts.com/mars/"
mars_table = pd.read_html(mars_facts_url)
mars_fact_df=mars_table[0]
mars_fact_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [18]:
# Setting columns and index
mars_fact_df.columns = ["Parameter", "Values"]
mars_fact_df= mars_fact_df.set_index(["Parameter"])
mars_fact_df

,Values
Parameter,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [19]:
# Converting to Html
mars_html_table = mars_fact_df.to_html()
mars_html_table = mars_html_table.replace("\n", "")
mars_html_table


'<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Parameter</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>      <td>Egyptian astronomers</td>    </tr>  </tbody></table>'

In [20]:
# OSX Users can run this to open the file in a browser, 
# or you can manually find the file and open it in the browser
!open mars_html_table.html

mars_dict["mars_html_table"] = mars_html_table


{'mar_weather': 'Sol 2142 (2018-08-15), high -10C/14F, low -71C/-95F, pressure at 8.65 hPa, daylight 05:28-17:41',
 'mars_html_table': '<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th></th>      <th>Values</th>    </tr>    <tr>      <th>Parameter</th>      <th></th>    </tr>  </thead>  <tbody>    <tr>      <th>Equatorial Diameter:</th>      <td>6,792 km</td>    </tr>    <tr>      <th>Polar Diameter:</th>      <td>6,752 km</td>    </tr>    <tr>      <th>Mass:</th>      <td>6.42 x 10^23 kg (10.7% Earth)</td>    </tr>    <tr>      <th>Moons:</th>      <td>2 (Phobos &amp; Deimos)</td>    </tr>    <tr>      <th>Orbit Distance:</th>      <td>227,943,824 km (1.52 AU)</td>    </tr>    <tr>      <th>Orbit Period:</th>      <td>687 days (1.9 years)</td>    </tr>    <tr>      <th>Surface Temperature:</th>      <td>-153 to 20 °C</td>    </tr>    <tr>      <th>First Record:</th>      <td>2nd millennium BC</td>    </tr>    <tr>      <th>Recorded By:</th>    

# Mars Hemispheres

In [416]:
import time 
hemispheres_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(hemispheres_url)
html = browser.html
soup = bs(html, "html.parser")
mars_hemisphere = []

products = soup.find("div", class_ = "result-list" )
hemispheres = products.find_all("div", class_="item")

for hemisphere in hemispheres:
    title = hemisphere.find("h3").text
    title = title.replace("Enhanced", "")
    end_link = hemisphere.find("a")["href"]
    image_link = "https://astrogeology.usgs.gov/" + end_link    
    browser.visit(image_link)
    html = browser.html
    soup=bs(html, "html.parser")
    downloads = soup.find("div", class_="downloads")
    image_url = downloads.find("a")["href"]
    mars_hemisphere.append({"title": title, "img_url": image_url})

In [417]:
mars_hemisphere

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
  'title': 'Cerberus Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
  'title': 'Schiaparelli Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
  'title': 'Syrtis Major Hemisphere '},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
  'title': 'Valles Marineris Hemisphere '}]

In [418]:
# mars_data={}
mars_data['mars_hemisphere'] = mars_hemisphere
mars_data

{'mars_hemisphere': [{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg',
   'title': 'Cerberus Hemisphere '},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg',
   'title': 'Schiaparelli Hemisphere '},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg',
   'title': 'Syrtis Major Hemisphere '},
  {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',
   'title': 'Valles Marineris Hemisphere '}]}

In [383]:
# mars_dict = {}
mars_dict["news_title"] = news_title
mars_dict["news_paragraph"] = news_paragraph
mars_dict

{'news_paragraph': 'A series of NASA videos highlight scientists and engineers leading the next mission to Mars.',
 'news_title': "Meet the People Behind NASA's InSight Mars Lander"}

In [478]:
mars_dict["profile_html"] = profile_html
mars_dict["mar_weather"]

'Sol 2108 (2018-07-12), Sunny, high -24C/-11F, low -65C/-84F, pressure at 8.06 hPa, daylight 05:19-17:27'